In [1]:
import pandas as pd
import numpy as np

#Lee la plantilla de fase de grupos
df=pd.read_csv('Fase Grupos.csv',sep=';',encoding='latin-1')
df

,Jornada,Grupo,Local,GL,GV,Visitante,DS,Fecha,Hora
0,J1,GRUPO A,Alemania,NaN,NaN,Escocia,V,14/06/2024,21:00
1,J1,GRUPO A,Hungría,NaN,NaN,Suiza,S,15/06/2024,15:00
2,J1,GRUPO B,España,NaN,NaN,Croacia,S,15/06/2024,18:00
3,J1,GRUPO B,Italia,NaN,NaN,Albania,S,15/06/2024,21:00
4,J1,GRUPO D,Polonia,NaN,NaN,Países Bajos,D,16/06/2024,15:00
5,J1,GRUPO C,Eslovenia,NaN,NaN,Dinamarca,D,16/06/2024,18:00
6,J1,GRUPO C,Serbia,NaN,NaN,Inglaterra,D,16/06/2024,21:00
7,J1,GRUPO E,Rumanía,NaN,NaN,Ucrania,L,17/06/2024,15:00
8,J1,GRUPO E,Bélgica,NaN,NaN,Eslovaquia,L,17/06/2024,18:00
9,J1,GRUPO D,Austria,NaN,NaN,Francia,L,17/06/2024,21:00


In [2]:
# Función para actualizar la clasificación basada en los resultados
def update_classification(results_df, classification_df):
    for index, row in results_df.iterrows():
        home_team = row['Local']
        away_team = row['Visitante']
        home_goals = row['GL']
        away_goals = row['GV']

        # Actualizar goles a favor y en contra
        classification_df.loc[classification_df['Selección'] == home_team, 'GF'] += home_goals
        classification_df.loc[classification_df['Selección'] == home_team, 'GC'] += away_goals
        classification_df.loc[classification_df['Selección'] == home_team, 'DG'] += home_goals-away_goals
        classification_df.loc[classification_df['Selección'] == away_team, 'GF'] += away_goals
        classification_df.loc[classification_df['Selección'] == away_team, 'GC'] += home_goals
        classification_df.loc[classification_df['Selección'] == away_team, 'DG'] += away_goals-home_goals

        # Actualizar puntos
        if home_goals > away_goals:
            classification_df.loc[classification_df['Selección'] == home_team, 'Pts'] += 3
        elif home_goals < away_goals:
            classification_df.loc[classification_df['Selección'] == away_team, 'Pts'] += 3
        else:
            classification_df.loc[classification_df['Selección'] == home_team, 'Pts'] += 1
            classification_df.loc[classification_df['Selección'] == away_team, 'Pts'] += 1
    return classification_df


# Función para calcular puntos de enfrentamientos directos
def direct_encounters_points(team1, team2, results_df):
    points = 0
    difgoals = 0
    favgoals = 0
    
    # Filtrar los partidos entre team1 y team2
    matches = results_df[((results_df['Local'] == team1) & (results_df['Visitante'] == team2)) | 
            ((results_df['Local'] == team2) & (results_df['Visitante'] == team1))]

    for _, match in matches.iterrows():
        if match['Local'] == team1:
            favgoals += match['GL']
            difgoals += match['GL']-match['GV']
            if match['GL'] > match['GV']:
                points += 3
            elif match['GL'] == match['GV']:
                points += 1
        elif match['Local'] == team2:
            favgoals += match['GV']
            difgoals += match['GV']-match['GL']
            if match['GL'] < match['GV']:
                points += 3
            elif match['GL'] == match['GV']:
                points += 1

    return points, difgoals, favgoals


# Función para actualizar la clasificación basada en los resultados
def order_classification(results_df, classification_df):
    classification_df = classification_df.sort_values(by=['Grupo', 'Pts'],ascending=[True,False]).reset_index(drop=True)
    for z in range(6):
        i = 0
        while i < 3:
            j = i
            while j < 4 and classification_df.at[z*4+j, 'Pts'] == classification_df.at[z*4+j + 1, 'Pts']:
                j += 1

            if j > i:
                # Equipos empatados entre las filas i y j
                teams_tied = classification_df.iloc[z*4+i:z*4+j+1].copy()
            
                # Calcular puntos de enfrentamientos directos
                points_direct = []
                difgoals_direct = []
                favgoals_direct = []
                for k in range(len(teams_tied)):
                    points_sum = 0
                    difgoals_sum = 0
                    favgoals_sum = 0
                    for l in range(len(teams_tied)):
                        if k != l:
                            points, difgoals, favgoals = direct_encounters_points(teams_tied.iloc[k]['Selección'], teams_tied.iloc[l]['Selección'], results_df)
                            points_sum += points
                            difgoals_sum += difgoals
                            favgoals_sum += favgoals
                    points_direct.append(points_sum)
                    difgoals_direct.append(difgoals_sum)
                    favgoals_direct.append(favgoals_sum)
                
            
                # Añadir columna de puntos de enfrentamientos directos y ordenar
                teams_tied['DirectPoints'] = points_direct
                teams_tied['DirectDifGoals'] = difgoals_direct
                teams_tied['DirectFavGoals'] = favgoals_direct
                teams_tied = teams_tied.sort_values(by=['DirectPoints','DirectDifGoals','DirectFavGoals', 'DG', 'GF'], ascending= False).drop(columns=['DirectPoints','DirectDifGoals','DirectFavGoals'])
            
                # Colocar equipos ordenados en el DataFrame final
                classification_df.iloc[z*4+i:z*4+j+1] = teams_tied.values
        
            i = j + 1
    
    classification_df['Pos.'] = [1,2,3,4]*6
    
    # Crear una lista de las columnas del DataFrame
    columnas = classification_df.columns.tolist()

    # Mover la nueva columna al principio de la lista
    columnas.insert(1, columnas.pop(columnas.index('Pos.')))

    # Reordenar el DataFrame con la nueva columna en la primera posición
    classification_df = classification_df[columnas]

    return classification_df

In [3]:
#Simulación aleatoria de la fase de grupos. No ejecutar de forma manual
num_filas = 36  # Número de filas en el DataFrame
df['GL'] = np.random.randint(1, 5, size=num_filas)  
df['GV'] = np.random.randint(1, 5, size=num_filas)  
df

,Jornada,Grupo,Local,GL,GV,Visitante,DS,Fecha,Hora
0,J1,GRUPO A,Alemania,1,2,Escocia,V,14/06/2024,21:00
1,J1,GRUPO A,Hungría,2,1,Suiza,S,15/06/2024,15:00
2,J1,GRUPO B,España,3,2,Croacia,S,15/06/2024,18:00
3,J1,GRUPO B,Italia,2,4,Albania,S,15/06/2024,21:00
4,J1,GRUPO D,Polonia,3,3,Países Bajos,D,16/06/2024,15:00
5,J1,GRUPO C,Eslovenia,1,1,Dinamarca,D,16/06/2024,18:00
6,J1,GRUPO C,Serbia,4,4,Inglaterra,D,16/06/2024,21:00
7,J1,GRUPO E,Rumanía,1,4,Ucrania,L,17/06/2024,15:00
8,J1,GRUPO E,Bélgica,2,2,Eslovaquia,L,17/06/2024,18:00
9,J1,GRUPO D,Austria,4,2,Francia,L,17/06/2024,21:00


In [4]:
# Elimina los valores duplicados
df_equipos = df[['Grupo','Local']].drop_duplicates()

#Renombra la columna de las selecciones
df_equipos.rename(columns={'Local': 'Selección'}, inplace=True)

# Ordena por grupo y equipo
df_equipos = df_equipos.sort_values(by=['Grupo', 'Selección']).reset_index(drop=True)

#Crea la tabla de clasificaciones
df_clas = df_equipos

#Añade los valores numéricos
df_clas[['Pts', 'GF', 'GC', 'DG']] = 0

# Actualizar la clasificación basada en los resultados
df_clas = update_classification(df, df_clas)

# Mostrar la clasificación actualizada
df_clas

,Grupo,Selección,Pts,GF,GC,DG
0,GRUPO A,Alemania,3,4,5,-1
1,GRUPO A,Escocia,4,4,6,-2
2,GRUPO A,Hungría,4,4,4,0
3,GRUPO A,Suiza,6,7,4,3
4,GRUPO B,Albania,9,11,6,5
5,GRUPO B,Croacia,3,7,8,-1
6,GRUPO B,España,6,7,7,0
7,GRUPO B,Italia,0,5,9,-4
8,GRUPO C,Dinamarca,5,8,5,3
9,GRUPO C,Eslovenia,1,4,9,-5


In [5]:
# Actualizar la clasificación basada en los resultados
df_clas = order_classification(df, df_clas)

# Mostrar la clasificación actualizada
df_clas

,Grupo,Pos.,Selección,Pts,GF,GC,DG
0,GRUPO A,1,Suiza,6,7,4,3
1,GRUPO A,2,Hungría,4,4,4,0
2,GRUPO A,3,Escocia,4,4,6,-2
3,GRUPO A,4,Alemania,3,4,5,-1
4,GRUPO B,1,Albania,9,11,6,5
5,GRUPO B,2,España,6,7,7,0
6,GRUPO B,3,Croacia,3,7,8,-1
7,GRUPO B,4,Italia,0,5,9,-4
8,GRUPO C,1,Serbia,5,11,8,3
9,GRUPO C,2,Dinamarca,5,8,5,3


In [6]:
#Guarda las clasificaciones
df_clas.to_csv('Clasificaciones Grupos.csv', index=False)

In [7]:
import pandas as pd

#Recupera las clasificaciones
df_clas=pd.read_csv('Clasificaciones Grupos.csv')

#Lee la plantilla de eliminatorias
df_elim=pd.read_csv('Fase Eliminatoria Plantilla.csv',sep=';',encoding='latin-1')

#Crea el dataframe de octavos
df_octavos=df_elim.head(8)
df_octavos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,81,OCTAVOS,2A,NaN,NaN,2B,NaN,NaN,NaN,S,29/06/2024,18:00
1,82,OCTAVOS,1A,NaN,NaN,2C,NaN,NaN,NaN,S,29/06/2024,21:00
2,83,OCTAVOS,1C,NaN,NaN,3D/E/F,NaN,NaN,NaN,D,30/06/2024,18:00
3,84,OCTAVOS,1B,NaN,NaN,3A/D/E/F,NaN,NaN,NaN,D,30/06/2024,21:00
4,85,OCTAVOS,2D,NaN,NaN,2E,NaN,NaN,NaN,L,01/07/2024,18:00
5,86,OCTAVOS,1F,NaN,NaN,3A/B/C,NaN,NaN,NaN,L,01/07/2024,21:00
6,87,OCTAVOS,1E,NaN,NaN,3A/B/C/D,NaN,NaN,NaN,M,02/07/2024,18:00
7,88,OCTAVOS,1D,NaN,NaN,2F,NaN,NaN,NaN,M,02/07/2024,21:00


In [8]:
#Selecciona los 3º de grupo
df_terceros=df_clas[df_clas['Pos.']==3]

#Ordena los 3º de grupo
df_terceros=df_terceros.sort_values(by=['Pts','DG','GF'],ascending=False).reset_index(drop=True)

#Escoge los 4 mejores
df_terceros=df_terceros.head(4)

#Escoge los grupos de los 4 mejores
terceros=df_terceros['Grupo'].unique()
df_terceros

,Grupo,Pos.,Selección,Pts,GF,GC,DG
0,GRUPO C,3,Inglaterra,4,9,10,-1
1,GRUPO D,3,Países Bajos,4,8,9,-1
2,GRUPO A,3,Escocia,4,4,6,-2
3,GRUPO B,3,Croacia,3,7,8,-1


In [9]:
#Realiza el seeding de los 4 mejores terceros
if 'GRUPO A' in terceros:
    if 'GRUPO B' in terceros:
        if 'GRUPO C' in terceros:
            tercero2='GRUPO A'
            tercero4='GRUPO B'
            tercero3='GRUPO C'
            if 'GRUPO D' in terceros:
                tercero1='GRUPO D'
            elif 'GRUPO E' in terceros:
                tercero1='GRUPO E'
            else:
                tercero1='GRUPO F'
        elif 'GRUPO D' in terceros:
            tercero2='GRUPO D'
            tercero4='GRUPO A'
            tercero3='GRUPO B'
            if 'GRUPO E' in terceros:
                tercero1='GRUPO E'
            else:
                tercero1='GRUPO F'
        else:
            tercero2='GRUPO E'
            tercero1='GRUPO F'
            tercero4='GRUPO B'
            tercero3='GRUPO A'
    elif 'GRUPO C' in terceros:
        tercero4='GRUPO C'
        tercero3='GRUPO A'
        if 'GRUPO D' in terceros:
            tercero1='GRUPO D'
            if 'GRUPO E' in terceros:
                tercero2='GRUPO E'
            else:
                tercero2='GRUPO F'
        else:
            tercero2='GRUPO E'
            tercero1='GRUPO F'
    else:
        tercero2='GRUPO E'
        tercero1='GRUPO F'
        tercero4='GRUPO D'
        tercero3='GRUPO A'
elif 'GRUPO B' in terceros:
    if 'GRUPO C' in terceros:
        if 'GRUPO D' in terceros:
            tercero1='GRUPO D'
            if 'GRUPO E' in terceros:
                tercero2='GRUPO E'
                tercero4='GRUPO B'
                tercero3='GRUPO C'
            else:
                tercero2='GRUPO F'
                tercero4='GRUPO C'
                tercero3='GRUPO B'
        else:
            tercero2='GRUPO F'
            tercero1='GRUPO E'
            tercero4='GRUPO C'
            tercero3='GRUPO B'
    else:
        tercero2='GRUPO F'
        tercero1='GRUPO E'
        tercero4='GRUPO D'
        tercero3='GRUPO B'
else:
    tercero2='GRUPO F'
    tercero1='GRUPO E'
    tercero4='GRUPO D'
    tercero3='GRUPO C'

In [10]:
# Diccionario para definir los cruces de octavos de final según la posición en los grupos
cruces = {
    81: {'Equipo1': ('GRUPO A', 2), 'Equipo2': ('GRUPO B', 2)},
    82: {'Equipo1': ('GRUPO A', 1), 'Equipo2': ('GRUPO C', 2)},
    83: {'Equipo1': ('GRUPO C', 1), 'Equipo2': (tercero1, 3)},
    84: {'Equipo1': ('GRUPO B', 1), 'Equipo2': (tercero2, 3)},
    85: {'Equipo1': ('GRUPO D', 2), 'Equipo2': ('GRUPO E', 2)},
    86: {'Equipo1': ('GRUPO F', 1), 'Equipo2': (tercero3, 3)},
    87: {'Equipo1': ('GRUPO E', 1), 'Equipo2': (tercero4, 3)},
    88: {'Equipo1': ('GRUPO D', 1), 'Equipo2': ('GRUPO F', 2)}
}

# Actualizar el DataFrame de octavos de final con los equipos correspondientes
for idx, row in df_octavos.iterrows():
    partido = row['Part.']
    equipo1_grupo, equipo1_posicion = cruces[partido]['Equipo1']
    equipo2_grupo, equipo2_posicion = cruces[partido]['Equipo2']
    
    equipo1 = df_clas[(df_clas['Grupo'] == equipo1_grupo) & (df_clas['Pos.'] == equipo1_posicion)]['Selección'].values[0]
    equipo2 = df_clas[(df_clas['Grupo'] == equipo2_grupo) & (df_clas['Pos.'] == equipo2_posicion)]['Selección'].values[0]
    
    df_octavos.at[idx, 'Local'] = equipo1
    df_octavos.at[idx, 'Visitante'] = equipo2

In [11]:
df_octavos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,81,OCTAVOS,Hungría,NaN,NaN,España,NaN,NaN,NaN,S,29/06/2024,18:00
1,82,OCTAVOS,Suiza,NaN,NaN,Dinamarca,NaN,NaN,NaN,S,29/06/2024,21:00
2,83,OCTAVOS,Serbia,NaN,NaN,Países Bajos,NaN,NaN,NaN,D,30/06/2024,18:00
3,84,OCTAVOS,Albania,NaN,NaN,Escocia,NaN,NaN,NaN,D,30/06/2024,21:00
4,85,OCTAVOS,Austria,NaN,NaN,Rumanía,NaN,NaN,NaN,L,01/07/2024,18:00
5,86,OCTAVOS,Portugal,NaN,NaN,Inglaterra,NaN,NaN,NaN,L,01/07/2024,21:00
6,87,OCTAVOS,Ucrania,NaN,NaN,Croacia,NaN,NaN,NaN,M,02/07/2024,18:00
7,88,OCTAVOS,Polonia,NaN,NaN,Georgia,NaN,NaN,NaN,M,02/07/2024,21:00


In [12]:
#Guarda los enfrentamientos de octavos
df_octavos.to_csv('Fase Eliminatoria Octavos.csv', index=False)

RELLENA LOS OCTAVOS!!

In [13]:
import pandas as pd
#Recupera los octavos
df_octavos=pd.read_csv('Fase Eliminatoria Octavos.csv')

#Lee la plantilla de eliminatorias
df_elim=pd.read_csv('Fase Eliminatoria Plantilla.csv',sep=';',encoding='latin-1')

#Crea el dataframe de cuartos
df_cuartos=df_elim[8:12]
df_cuartos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
8,41,CUARTOS,G84,NaN,NaN,G82,NaN,NaN,NaN,V,05/07/2024,18:00
9,42,CUARTOS,G86,NaN,NaN,G85,NaN,NaN,NaN,V,05/07/2024,21:00
10,43,CUARTOS,G83,NaN,NaN,G81,NaN,NaN,NaN,S,06/07/2024,18:00
11,44,CUARTOS,G87,NaN,NaN,G88,NaN,NaN,NaN,S,06/07/2024,21:00


In [14]:
#Simulación aleatoria de los octavos. No ejecutar de forma manual
num_filas = 8  # Número de filas en el DataFrame
df_octavos['GL']= np.random.randint(0, 5, size=num_filas) 
df_octavos['GV']= np.random.randint(0, 5, size=num_filas) 
for idx, row in df_octavos.iterrows():
    if row['GL'] == row['GV']:
        df_octavos.at[idx, 'GPrL'] = np.random.randint(row['GL'], row['GL']+2) 
        df_octavos.at[idx, 'GPrV'] = np.random.randint(row['GV'], row['GV']+2)
        if df_octavos.at[idx, 'GPrL'] == df_octavos.at[idx, 'GPrV']:
            df_octavos.at[idx, 'WPn'] = np.random.randint(1, 3)

df_octavos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,81,OCTAVOS,Hungría,4,1,España,NaN,NaN,NaN,S,29/06/2024,18:00
1,82,OCTAVOS,Suiza,4,2,Dinamarca,NaN,NaN,NaN,S,29/06/2024,21:00
2,83,OCTAVOS,Serbia,3,4,Países Bajos,NaN,NaN,NaN,D,30/06/2024,18:00
3,84,OCTAVOS,Albania,0,1,Escocia,NaN,NaN,NaN,D,30/06/2024,21:00
4,85,OCTAVOS,Austria,2,2,Rumanía,3.0,3.0,1.0,L,01/07/2024,18:00
5,86,OCTAVOS,Portugal,0,0,Inglaterra,0.0,0.0,1.0,L,01/07/2024,21:00
6,87,OCTAVOS,Ucrania,0,3,Croacia,NaN,NaN,NaN,M,02/07/2024,18:00
7,88,OCTAVOS,Polonia,2,3,Georgia,NaN,NaN,NaN,M,02/07/2024,21:00


In [15]:
# Diccionario para definir los cruces de octavos de final según los octavos
cruces = {
    41: {'Equipo1': (84), 'Equipo2': (82)},
    42: {'Equipo1': (86), 'Equipo2': (85)},
    43: {'Equipo1': (83), 'Equipo2': (81)},
    44: {'Equipo1': (87), 'Equipo2': (88)},
}

# Actualizar el DataFrame de cuartos de final con los equipos correspondientes
for idx, row in df_cuartos.iterrows():
    partido = row['Part.']
    equipo1_partido = cruces[partido]['Equipo1']
    equipo2_partido = cruces[partido]['Equipo2']

    if df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GL'].values[0] > df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GV'].values[0]:
        equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Local'].values[0]
    elif df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GL'].values[0] < df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GV'].values[0]:
        equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Visitante'].values[0]
    else:
        if df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GPrL'].values[0] > df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GPrV'].values[0]:
            equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Local'].values[0]
        elif df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GPrL'].values[0] < df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['GPrV'].values[0]:
            equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Visitante'].values[0]
        else:
            if df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['WPn'].values[0]==1:
                equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Local'].values[0]
            else:
                equipo1 = df_octavos[(df_octavos['Part.'] == equipo1_partido) ]['Visitante'].values[0]


    if df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GL'].values[0] > df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GV'].values[0]:
        equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Local'].values[0]
    elif df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GL'].values[0] < df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GV'].values[0]:
        equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
    else:
        if df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GPrL'].values[0] > df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GPrV'].values[0]:
            equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Local'].values[0]
        elif df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GPrL'].values[0] < df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['GPrV'].values[0]:
            equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
        else:
            if df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['WPn'].values[0]==1:
                equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Local'].values[0]
            else:
                equipo2 = df_octavos[(df_octavos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
    
    df_cuartos.at[idx, 'Local'] = equipo1
    df_cuartos.at[idx, 'Visitante'] = equipo2

df_cuartos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
8,41,CUARTOS,Escocia,NaN,NaN,Suiza,NaN,NaN,NaN,V,05/07/2024,18:00
9,42,CUARTOS,Portugal,NaN,NaN,Austria,NaN,NaN,NaN,V,05/07/2024,21:00
10,43,CUARTOS,Países Bajos,NaN,NaN,Hungría,NaN,NaN,NaN,S,06/07/2024,18:00
11,44,CUARTOS,Croacia,NaN,NaN,Georgia,NaN,NaN,NaN,S,06/07/2024,21:00


In [16]:
#Guarda los enfrentamientos de cuartos
df_cuartos.to_csv('Fase Eliminatoria Cuartos.csv', index=False)

RELLENA LOS CUARTOS!!

In [17]:
import pandas as pd

#Recupera los cuartos
df_cuartos=pd.read_csv('Fase Eliminatoria Cuartos.csv')

#Lee la plantilla de eliminatorias
df_elim=pd.read_csv('Fase Eliminatoria Plantilla.csv',sep=';',encoding='latin-1')

#Crea el dataframe de semis
df_semis=df_elim[12:14]
df_semis

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
12,21,SEMIFINAL,G41,NaN,NaN,G42,NaN,NaN,NaN,M,09/07/2024,21:00
13,22,SEMIFINAL,G44,NaN,NaN,G43,NaN,NaN,NaN,X,10/07/2024,21:00


In [18]:
#Simulación aleatoria de los cuartos. No ejecutar de forma manual
num_filas = 4  # Número de filas en el DataFrame
df_cuartos['GL']= np.random.randint(0, 5, size=num_filas) 
df_cuartos['GV']= np.random.randint(0, 5, size=num_filas) 
for idx, row in df_cuartos.iterrows():
    if row['GL'] == row['GV']:
        df_cuartos.at[idx, 'GPrL'] = np.random.randint(row['GL'], row['GL']+2) 
        df_cuartos.at[idx, 'GPrV'] = np.random.randint(row['GV'], row['GV']+2)
        if df_cuartos.at[idx, 'GPrL'] == df_cuartos.at[idx, 'GPrV']:
            df_cuartos.at[idx, 'WPn'] = np.random.randint(1, 3)
df_cuartos

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,41,CUARTOS,Escocia,2,0,Suiza,NaN,NaN,NaN,V,05/07/2024,18:00
1,42,CUARTOS,Portugal,1,4,Austria,NaN,NaN,NaN,V,05/07/2024,21:00
2,43,CUARTOS,Países Bajos,1,0,Hungría,NaN,NaN,NaN,S,06/07/2024,18:00
3,44,CUARTOS,Croacia,2,0,Georgia,NaN,NaN,NaN,S,06/07/2024,21:00


In [19]:
# Diccionario para definir los cruces de semifinales según la posición en los grupos
cruces = {
    21: {'Equipo1': (41), 'Equipo2': (42)},
    22: {'Equipo1': (44), 'Equipo2': (43)},
}


# Actualizar el DataFrame de semifinales con los equipos correspondientes
for idx, row in df_semis.iterrows():
    partido = row['Part.']
    equipo1_partido = cruces[partido]['Equipo1']
    equipo2_partido = cruces[partido]['Equipo2']

    if df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GL'].values[0] > df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GV'].values[0]:
        equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Local'].values[0]
    elif df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GL'].values[0] < df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GV'].values[0]:
        equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Visitante'].values[0]
    else:
        if df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GPrL'].values[0] > df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GPrV'].values[0]:
            equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Local'].values[0]
        elif df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GPrL'].values[0] < df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['GPrV'].values[0]:
            equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Visitante'].values[0]
        else:
            if df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['WPn'].values[0]==1:
                equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Local'].values[0]
            else:
                equipo1 = df_cuartos[(df_cuartos['Part.'] == equipo1_partido) ]['Visitante'].values[0]


    if df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GL'].values[0] > df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GV'].values[0]:
        equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Local'].values[0]
    elif df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GL'].values[0] < df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GV'].values[0]:
        equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
    else:
        if df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GPrL'].values[0] > df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GPrV'].values[0]:
            equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Local'].values[0]
        elif df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GPrL'].values[0] < df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['GPrV'].values[0]:
            equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
        else:
            if df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['WPn'].values[0]==1:
                equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Local'].values[0]
            else:
                equipo2 = df_cuartos[(df_cuartos['Part.'] == equipo2_partido) ]['Visitante'].values[0]
    
    df_semis.at[idx, 'Local'] = equipo1
    df_semis.at[idx, 'Visitante'] = equipo2

df_semis

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
12,21,SEMIFINAL,Escocia,NaN,NaN,Austria,NaN,NaN,NaN,M,09/07/2024,21:00
13,22,SEMIFINAL,Croacia,NaN,NaN,Países Bajos,NaN,NaN,NaN,X,10/07/2024,21:00


In [20]:
#Guarda los enfrentamientos de semis
df_semis.to_csv('Fase Eliminatoria Semis.csv', index=False)

RELLENA LAS SEMIS!!

In [21]:
import pandas as pd
#Recupera las semis
df_semis=pd.read_csv('Fase Eliminatoria Semis.csv')

#Lee la plantilla de eliminatorias
df_elim=pd.read_csv('Fase Eliminatoria Plantilla.csv',sep=';',encoding='latin-1')

#Crea el dataframe de la final
df_final=df_elim.tail(1)
df_final

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
14,11,FINAL,G21,NaN,NaN,G22,NaN,NaN,NaN,D,14/07/2024,21:00


In [22]:
#Simulación aleatoria de las semis. No ejecutar de forma manual
num_filas = 2 # Número de filas en el DataFrame
df_semis['GL']= np.random.randint(0, 5, size=num_filas) 
df_semis['GV']= np.random.randint(0, 5, size=num_filas)
for idx, row in df_semis.iterrows():
    if row['GL'] == row['GV']:
        df_semis.at[idx, 'GPrL'] = np.random.randint(row['GL'], row['GL']+2) 
        df_semis.at[idx, 'GPrV'] = np.random.randint(row['GV'], row['GV']+2)
        if df_semis.at[idx, 'GPrL'] == df_semis.at[idx, 'GPrV']:
            df_semis.at[idx, 'WPn'] = np.random.randint(1, 3)
df_semis

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,21,SEMIFINAL,Escocia,1,3,Austria,NaN,NaN,NaN,M,09/07/2024,21:00
1,22,SEMIFINAL,Croacia,4,3,Países Bajos,NaN,NaN,NaN,X,10/07/2024,21:00


In [23]:
# Actualizar el DataFrame de la final con los equipos correspondientes
if df_semis[(df_semis['Part.'] == 21) ]['GL'].values[0] > df_semis[(df_semis['Part.'] == 21) ]['GV'].values[0]:
    equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Local'].values[0]
elif df_semis[(df_semis['Part.'] == 21) ]['GL'].values[0] < df_semis[(df_semis['Part.'] == 21) ]['GV'].values[0]:
    equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Visitante'].values[0]
else:
    if df_semis[(df_semis['Part.'] == 21) ]['GPrL'].values[0] > df_semis[(df_semis['Part.'] == 21) ]['GPrV'].values[0]:
        equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Local'].values[0]
    elif df_semis[(df_semis['Part.'] == 21) ]['GPrL'].values[0] < df_semis[(df_semis['Part.'] == 21) ]['GPrV'].values[0]:
        equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Visitante'].values[0]
    else:
        if df_semis[(df_semis['Part.'] == 21) ]['WPn'].values[0]==1:
            equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Local'].values[0]
        else:
            equipo1 = df_semis[(df_semis['Part.'] == 21) ]['Visitante'].values[0]


if df_semis[(df_semis['Part.'] == 22) ]['GL'].values[0] > df_semis[(df_semis['Part.'] == 22) ]['GV'].values[0]:
    equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Local'].values[0]
elif df_semis[(df_semis['Part.'] == 22) ]['GL'].values[0] < df_semis[(df_semis['Part.'] == 22) ]['GV'].values[0]:
        equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Visitante'].values[0]
else:
    if df_semis[(df_semis['Part.'] == 22) ]['GPrL'].values[0] > df_semis[(df_semis['Part.'] == 22) ]['GPrV'].values[0]:
        equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Local'].values[0]
    elif df_semis[(df_semis['Part.'] == 22) ]['GPrL'].values[0] < df_semis[(df_semis['Part.'] == 22) ]['GPrV'].values[0]:
        equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Visitante'].values[0]
    else:
        if df_semis[(df_semis['Part.'] == 22) ]['WPn'].values[0]==1:
            equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Local'].values[0]
        else:
            equipo2 = df_semis[(df_semis['Part.'] == 22) ]['Visitante'].values[0]

df_final.at[14,'Local'] = equipo1
df_final.at[14,'Visitante'] = equipo2

df_final

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
14,11,FINAL,Austria,NaN,NaN,Croacia,NaN,NaN,NaN,D,14/07/2024,21:00


In [24]:
#Guarda la final
df_final.to_csv('Fase Eliminatoria Final.csv', index=False)

In [25]:
import pandas as pd
#Recupera la final
df_final=pd.read_csv('Fase Eliminatoria Final.csv')
df_final

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,11,FINAL,Austria,NaN,NaN,Croacia,NaN,NaN,NaN,D,14/07/2024,21:00


In [26]:
#Simulación aleatoria de las semis. No ejecutar de forma manual
df_final['GL']= np.random.randint(0, 5) 
df_final['GV']= np.random.randint(0, 5) 

if df_final.at[0, 'GL'] == df_final.at[0, 'GV']:
    df_final.at[0, 'GPrL'] = np.random.randint(df_final.at[0, 'GL'], df_final.at[0, 'GL'] + 2) 
    df_final.at[0, 'GPrV'] = np.random.randint(df_final.at[0, 'GV'], df_final.at[0, 'GV'] + 2)
    if df_final.at[0, 'GPrL'] == df_final.at[0, 'GPrV']:
        df_final.at[0, 'WPn'] = np.random.randint(1, 3)
df_final

,Part.,Fase,Local,GL,GV,Visitante,GPrL,GPrV,WPn,DS,Fecha,Hora
0,11,FINAL,Austria,3,1,Croacia,NaN,NaN,NaN,D,14/07/2024,21:00
